### Importing necessary librays for the programm to work

In [2]:
import pandas as pd 

#### Definng the functions ####

In [17]:
class Selic_interest():
    """
    Funcion created to calculate
    the profit with capital invested in 
    SELIC Rate of compounded interest from an given date
    until an end date.
    """


    def __init__(self):
        data = pd.read_json("https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=json&dataInicial=dd/mm/yy&dataFinal=dd/mm/yy")  # Reads the data from the API
        data.columns = ['Date', 'SELIC']   # Renaming the columns
        data.index = data['Date']  # Setting the index
        data.index = pd.to_datetime(data.index, dayfirst = True) # Transforming index into date time
        self.data = data    
    
    def set_parameters(self, start_date, end_date, capital, frequency: str = 'daily'):
        """
        Function to set the parameters for the analysis. Returns an dataframe with profit and capital evolution during the analysis.

        Parameters
        -----------
        start_date: `str` --> Initial date for the analysis. Specificy format: %Y/%M/%D
        end_date: `str` --> Final date for the analysis. Specificy format: %Y/%M/%D
        capital: `float` --> Capital invested in the analysis.
        frequency: `str` --> Frequency of the analysis. Default is daily.
        
        """
        if not isinstance(end_date, (str)):
            raise ValueError("End date must be a string with the following format: %Y/%M/%D")

        if not isinstance(start_date, (str)):
            raise ValueError("Start date must be a string with the following format: %Y/%M/%D")

        if not isinstance(capital, (int, float)):
            raise ValueError("Capital must be numeric")

        if not isinstance(frequency, (str)):
            raise ValueError("Frequency must be a string")

        if frequency not in ['daily', 'weekly', 'monthly', 'yearly']:
            raise ValueError("Frequency must be one of the following: daily, weekly, monthly, yearly")

        self.start_date = start_date
        self.end_date = end_date
        self.capital = capital
        self.frequency = frequency.lower()
    
        self.capital = [capital]
        self.get_selic_frame = self.data.loc[start_date : end_date]
        self.get_daily_selic = ((self.get_selic_frame.loc[:,['SELIC']] + 1) ** (1/252)) - 1   ### Transforming yearly selic to daily selic, so we can calculate the daily amount earned
        self.get_daily_selic.columns = ['Daily Selic']  ### Reanimg the columns
    
        [self.capital.append((self.capital[i-1]*(1+self.get_daily_selic['Daily Selic'][i-1]))) for i in range(1,len(self.get_daily_selic))] ### Calculating the daily amount earned with a list comprehension
    
        self.capital = pd.DataFrame(self.capital, index = self.get_daily_selic.index, columns = ['Capital']).merge(self.get_daily_selic, left_index = True, right_index = True)   ### merging the datasets
        self.capital['Amount Earned'] = self.capital['Capital'].diff()   ### calculating the amount earned between 2 dats
        self.capital['Amount Earned'][0] = 0   ### setting the first amount earned to 0, because we have no profit in the first day

        if frequency == 'daily':
            self.amount_earned = self.capital['Amount Earned'].cumsum() ### cumulative sum of the amount earned
            self.capital = self.capital['Capital']

        elif frequency == 'monthly':
            self.amount_earned = self.capital['Amount Earned'].resample('BM').agg(lambda x: x.sum()).cumsum() ### resampling the amount earned possessed to monthly frequency
            self.capital = self.capital.resample('M').last()['Capital']  ### resampling the capital possessed to monthly frequency

        elif frequency == 'weekly':
            self.amount_earned = self.capital['Amount Earned'].resample('W').agg(lambda x: x.sum()).cumsum() ### resampling the amount earned to weekly frequency
            self.capital = self.capital.resample('W').last()['Capital'] ### resampling the capital possessed to weekly frequency

        elif frequency == 'yearly':
            self.amount_earned = self.capital['Amount Earned'].resample('Y').agg(lambda x: x.sum()).cumsum() ### resampling the amount earned to yearly frequency
            self.capital = self.capital.resample('Y').last()['Capital'] ### resampling the capital possessed to yearly frequency

        return pd.merge(self.capital, self.amount_earned, left_index = True, right_index = True)


    def best_period(self,start_date, end_date, frequency:int = 500):
        """
        Function to get the best period of investment from an initial until an end date for an given frequency.

        Parameters
        -----------
        start_date: `str` --> Initial date for the analysis. Specificy format: %Y/%M/%D
        end_date: `str` --> Final date for the analysis. Specificy format: %Y/%M/%D
        frequency: `int` --> Frequency of the analysis. Default is 500.
        
        """
        if not isinstance(start_date, (str)):
            raise ValueError("Start date must be a string with the following format: %Y/%M/%D")
        if not isinstance(end_date, (str)):
             raise ValueError("End date must be a string with the following format: %Y/%M/%D")
        if not isinstance(frequency, (int)):
            raise ValueError("Frequency must be a integer")
        
        get_raw_data = self.data.loc[start_date : end_date]  
        get_raw_data = ((get_raw_data.loc[:,['SELIC']] + 1) ** (1/252)) - 1
        get_raw_data.columns = ['Daily Selic']
    
        get_rolling_windows = get_raw_data.rolling(frequency).agg(lambda x:((x + 1).prod()) - 1)
        best_period = get_rolling_windows[get_rolling_windows['Daily Selic'] == get_rolling_windows['Daily Selic'].max()]
        initial_date = str((pd.to_datetime(self.data.iloc[self.data.index.get_loc(str(best_period.index.format())[2:-2]) - frequency]['Date'], format = '%d/%m/%Y')))[0:-9]
        final_date = str(best_period.index.format())[2:-2]
        print(f'The best period was from {initial_date} to {final_date} with a cumulative return of {round(100 * best_period.iloc[0,0],4)}%')

In [18]:
function = Selic_interest()

In [12]:
function.set_parameters(start_date = '2010-01-01', end_date = '2022-03-01', capital = 12.20828, frequency = 'weekly')

,Capital,Amount Earned
Date,,
2010-01-10,12.214559,0.006279
2010-01-17,12.222413,0.014133
2010-01-24,12.230272,0.021992
2010-01-31,12.238135,0.029855
2010-02-07,12.246004,0.037724
...,...,...
2022-01-30,18.128287,5.920007
2022-02-06,18.140954,5.932674
2022-02-13,18.155135,5.946855


In [7]:
function.best_period('2000-01-01', '2022-03-01', 200)

The best period was from 2002-11-20 to 2003-09-08 with a cumulative return of 6.9784%
